In [ ]:
# --- IMPORTS ---
import arcpy, os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FuncFormatter

# --- GLOBAL FONT & LABEL SIZE SETTINGS (optional) ---
plt.rcParams.update({
    "axes.labelsize": 16,
    "xtick.labelsize": 13,
    "ytick.labelsize": 13,
    "legend.fontsize": 12
})

# --- USER INPUTS (EDIT THESE) ---
GDB = r"D:\GIS\REEF_ISLAND_RESEARCH.gdb"   # <<< EDIT if needed

SITE_FC = [
    ("podcaddi_rates_update", "Podang Caddi"),
    ("podlom_rates_update",   "Podang Lompo"),
    ("kodkeke_rates_update",  "Kodingareng Keke"),
]

WLR_FIELD = "WLR"   # weighted linear regression rate field

ACCRETION_THR =  0.1
EROSION_THR   = -0.1

OUT_PNG = r"D:\GIS\plots\WLR_boxplot_with_3maps.png"  # <<< EDIT

# RGB basemap rasters for the three mapped sites (EDIT paths)
RGB_RASTERS = {
    "podcaddi_rates_update": r"D:\GIS\Orthomosaic_spermonde\PODANG_CADDI\PODANG CADDI_transparent_mosaic_group1.tif",
    "podlom_rates_update":   r"D:\GIS\Orthomosaic_spermonde\PODANG_LOMPO\podang lompo_transparent_mosaic_group1.tif",
    "kodkeke_rates_update":  r"D:\GIS\Orthomosaic_spermonde\KODINGARENG KEKE\kodinagereng keke_transparent_mosaic_group1.tif"
}

# --- MAP SCALE CONTROL -------------------------------------------------
# All maps will use the same square window.
# max_span is computed; MAP_SCALE lets you zoom in/out:
#   MAP_SCALE = 1.0  -> full auto extent
#   MAP_SCALE < 1.0  -> zoom in (tighter window)
#   MAP_SCALE > 1.0  -> zoom out
MAP_SCALE = 0.65  # you found 0.65 optimal

# Optional per-site nudges (in metres) to fine-tune centring
# Positive dx = shift east, positive dy = shift north
MANUAL_CENTER_OFFSETS = {
    "podcaddi_rates_update": (0, 0),
    "podlom_rates_update":   (0, 0),
    "kodkeke_rates_update":  (0, 0),
}
# --------------------------------------------------------------------
arcpy.env.workspace = GDB
np.random.seed(42)

# --- HELPER: metres -> km for axis tick labels ---
def m_to_km(x, pos):
    return f"{x / 1000.0:.1f}"

km_formatter = FuncFormatter(m_to_km)

# --- COLLECT DATA ---

all_wlr = []
all_class = []

for fc_name, label in SITE_FC:
    fc_path = os.path.join(GDB, fc_name)
    if not arcpy.Exists(fc_path):
        raise RuntimeError(f"Feature class not found: {fc_path}")

    vals, cats = [], []
    with arcpy.da.SearchCursor(fc_path, [WLR_FIELD]) as cur:
        for (wlr,) in cur:
            if wlr is None:
                continue
            vals.append(float(wlr))
            if wlr > ACCRETION_THR:
                cats.append("Accretion")
            elif wlr < EROSION_THR:
                cats.append("Erosion")
            else:
                cats.append("No significant change")

    if not vals:
        raise RuntimeError(f"No non-null {WLR_FIELD} values in {fc_name}")

    all_wlr.append(np.array(vals))
    all_class.append(np.array(cats))

print("Collected WLR values for", len(all_wlr), "sites.")

# --- SUMMARY STATS PER SITE (for text in Results section) --------------

print("\n=== Shoreline-change summary by site ===")
for (fc_name, label), wlr_vals, classes in zip(SITE_FC, all_wlr, all_class):
    wlr_vals = np.array(wlr_vals)
    classes  = np.array(classes)

    n_total = len(wlr_vals)
    n_acc   = np.sum(classes == "Accretion")
    n_ero   = np.sum(classes == "Erosion")
    n_nc    = np.sum(classes == "No significant change")

    p_acc = 100 * n_acc / n_total
    p_ero = 100 * n_ero / n_total
    p_nc  = 100 * n_nc / n_total

    mu   = wlr_vals.mean()
    sigma = wlr_vals.std(ddof=1)

    print(f"\n{label}")
    print(f"  n transects        = {n_total}")
    print(f"  mean WLR           = {mu:.2f} m/yr")
    print(f"  sd WLR             = {sigma:.2f} m/yr")
    print(f"  Erosion            = {n_ero} transects ({p_ero:.1f}%)")
    print(f"  Accretion          = {n_acc} transects ({p_acc:.1f}%)")
    print(f"  No significant chg = {n_nc} transects ({p_nc:.1f}%)")



# --- STATS FOR LABELS ---
means = [arr.mean() for arr in all_wlr]
stds  = [arr.std(ddof=1) for arr in all_wlr]

# --- COLOURS ---
COLOR_ACCRETION = "#1C9AB7"   # positive WLR
COLOR_NOCHANGE  = "lightgrey"
COLOR_EROSION   = "tomato"    # negative WLR

CLASS_COLORS = {
    "Accretion": COLOR_ACCRETION,
    "No significant change": COLOR_NOCHANGE,
    "Erosion": COLOR_EROSION,
}

# --- FIGURE LAYOUT: boxplot + 3 maps ----------------------------------
fig = plt.figure(figsize=(12, 10))  # taller to fit 3 maps
gs = GridSpec(
    nrows=3,
    ncols=2,
    width_ratios=[1.4, 1.0],   # left wider for boxplot
    height_ratios=[1.0, 1.0, 1.0],
    figure=fig
)

ax_box          = fig.add_subplot(gs[:, 0])  # left column (all rows)
ax_map_podcaddi = fig.add_subplot(gs[0, 1])
ax_map_podlom   = fig.add_subplot(gs[1, 1])
ax_map_kodkeke  = fig.add_subplot(gs[2, 1])

x_positions = np.arange(1, len(SITE_FC) + 1)

# === BOX PLOT SECTION ==================================================

bp = ax_box.boxplot(
    all_wlr,
    positions=x_positions,
    widths=0.4,
    vert=True,
    patch_artist=True,
    showfliers=False,
)

for box in bp["boxes"]:
    box.set(facecolor="white", edgecolor="black", linewidth=1)
for median in bp["medians"]:
    median.set(color="black", linewidth=1.3)
for whisker in bp["whiskers"]:
    whisker.set(color="black", linewidth=1)
for cap in bp["caps"]:
    cap.set(color="black", linewidth=1)

# μ / σ / n text beside each box (aligned with median)
x_offset = 0.33  # horizontal offset to the right
for x, wlr_vals, mu, sigma in zip(x_positions, all_wlr, means, stds):
    median_y = np.median(wlr_vals)
    n = len(wlr_vals)
    text = f"μ = {mu:.2f}\nσ = {sigma:.2f}\nn = {n}"
    ax_box.text(
        x + x_offset,
        median_y,
        text,
        ha="left",
        va="center",
        fontsize=10
    )

# Jittered points
jitter_width = 0.15
point_size = 10

for i, (x, wlr_vals, classes) in enumerate(zip(x_positions, all_wlr, all_class)):
    xs = x + np.random.uniform(-jitter_width, jitter_width, size=len(wlr_vals))
    colors = [CLASS_COLORS[c] for c in classes]
    ax_box.scatter(xs, wlr_vals, s=point_size, c=colors,
                   alpha=0.8, edgecolors="none", zorder=3)

# Axis labels
ax_box.set_xticks(x_positions)
ax_box.set_xlim(0.5, len(SITE_FC) + 1.0)
ax_box.set_xticklabels([lbl for _, lbl in SITE_FC])
ax_box.set_ylabel("Weighted Linear Regression Rates (m/year)")

# --- Y-limits using all points ---
CLIP_OUTLIERS = False
CLIP_PERCENTILES = (1, 99)

all_concat = np.concatenate(all_wlr)

if CLIP_OUTLIERS:
    y_min, y_max = np.percentile(all_concat, CLIP_PERCENTILES)
else:
    y_min = all_concat.min()
    y_max = all_concat.max()

y_range = (y_max - y_min) if y_max > y_min else 1.0
ax_box.set_ylim(y_min - 0.1 * y_range, y_max + 0.1 * y_range)

# Zero line
ax_box.axhline(0, color="0.5", linestyle="--", linewidth=1, zorder=1)

# Legend
legend_elements = [
    Line2D([0], [0], marker="o", linestyle="None",
           color=COLOR_ACCRETION, label="Accretion", markersize=6),
    Line2D([0], [0], marker="o", linestyle="None",
           color=COLOR_NOCHANGE, label="No significant change", markersize=6),
    Line2D([0], [0], marker="o", linestyle="None",
           color=COLOR_EROSION, label="Erosion", markersize=6),
]
ax_box.legend(handles=legend_elements, loc="upper left", frameon=False)

# --- PANEL LETTERS (a–d) ----------------------------------------------
# upper-right of each axes (axes coords 0–1)
ax_box.text(0.98, 0.98, "a", transform=ax_box.transAxes,
            ha="right", va="top", fontsize=16, color="black")
ax_map_podcaddi.text(0.98, 0.98, "b", transform=ax_map_podcaddi.transAxes,
                     ha="right", va="top", fontsize=16, color="black")
ax_map_podlom.text(0.98, 0.98, "c", transform=ax_map_podlom.transAxes,
                   ha="right", va="top", fontsize=16, color="black")
ax_map_kodkeke.text(0.98, 0.98, "d", transform=ax_map_kodkeke.transAxes,
                    ha="right", va="top", fontsize=16, color="black")

# --- PRECOMPUTE COMMON MAP WINDOW (same scale for all maps) -----------

MAP_CENTERS = {}     # fc_name -> (cx, cy) based on transects
max_span = 0.0       # biggest width/height among sites

MAP_SITES = [
    "podcaddi_rates_update",
    "podlom_rates_update",
    "kodkeke_rates_update"
]

for fc_name in MAP_SITES:
    fc_path = os.path.join(GDB, fc_name)
    fc_ext = arcpy.Describe(fc_path).extent  # extent of transect lines

    # centre from transects (so island/transects sit in the middle)
    cx = (fc_ext.XMin + fc_ext.XMax) / 2.0
    cy = (fc_ext.YMin + fc_ext.YMax) / 2.0

    # apply optional manual offsets (in metres)
    dx, dy = MANUAL_CENTER_OFFSETS.get(fc_name, (0, 0))
    cx += dx
    cy += dy
    MAP_CENTERS[fc_name] = (cx, cy)

    # work out span using the larger of raster vs transect extents
    raster_path = RGB_RASTERS.get(fc_name, "")
    if raster_path and arcpy.Exists(raster_path):
        rast_ext = arcpy.Raster(raster_path).extent
        span_x = max(fc_ext.XMax - fc_ext.XMin, rast_ext.XMax - rast_ext.XMin)
        span_y = max(fc_ext.YMax - fc_ext.YMin, rast_ext.YMax - rast_ext.YMin)
    else:
        span_x = fc_ext.XMax - fc_ext.XMin
        span_y = fc_ext.YMax - fc_ext.YMin

    span = max(span_x, span_y)  # square window
    max_span = max(max_span, span)

# square window (same in x and y) so all maps have identical scale
MAP_HALF_SPAN = (max_span * MAP_SCALE) / 2.0


# === MAP SECTION =======================================================

def plot_site_map(ax, fc_name, site_label):
    """Plot RGB basemap + WLR-coloured transects for a given site."""
    fc_path = os.path.join(GDB, fc_name)
    raster_path = RGB_RASTERS.get(fc_name, "")

    # --- basemap (if provided) ---
    if raster_path and arcpy.Exists(raster_path):
        rast = arcpy.Raster(raster_path)
        arr = arcpy.RasterToNumPyArray(rast)  # shape: (bands, rows, cols)
        if arr.ndim == 3 and arr.shape[0] in (3, 4):  # 3 or 4 bands

            # --- Convert to RGB ---
            rgb = np.moveaxis(arr[:3, :, :], 0, -1)  # (rows, cols, bands)

            # --- REMOVE BLACK BACKGROUND ------------------------------------
            # Black background in orthomosaics is usually [0,0,0].
            alpha = np.where(
                (rgb[:, :, 0] == 0) &
                (rgb[:, :, 1] == 0) &
                (rgb[:, :, 2] == 0),
                0, 255
            ).astype(np.uint8)

            rgba = np.dstack((rgb, alpha))
            # -----------------------------------------------------------------

            extent = [
                rast.extent.XMin, rast.extent.XMax,
                rast.extent.YMin, rast.extent.YMax
            ]

            ax.imshow(rgba, extent=extent, origin="upper")

        else:
            print(f"Raster {raster_path} is not a 3-band RGB image.")
    else:
        print(f"No RGB raster found/defined for {fc_name}, plotting transects only.")

    # --- WLR-coloured transects ---
    with arcpy.da.SearchCursor(fc_path, [WLR_FIELD, "SHAPE@"]) as cur:
        for wlr, geom in cur:
            if wlr is None or geom is None:
                continue

            if wlr > 0:
                col = COLOR_ACCRETION
            elif wlr < 0:
                col = COLOR_EROSION
            else:
                col = COLOR_NOCHANGE

            for part in geom:
                xs = [pt.X for pt in part if pt]
                ys = [pt.Y for pt in part if pt]
                if len(xs) >= 2:
                    ax.plot(xs, ys, color=col, linewidth=1.0, alpha=0.9)

    # --- common window / scale for all maps ---
    cx, cy = MAP_CENTERS[fc_name]
    half = MAP_HALF_SPAN

    ax.set_xlim(cx - half, cx + half)
    ax.set_ylim(cy - half, cy + half)

    ax.set_title(site_label)
    ax.set_xlabel("Easting (km)")
    ax.set_ylabel("Northing (km)")

    # Format ticks as km
    ax.xaxis.set_major_formatter(km_formatter)
    ax.yaxis.set_major_formatter(km_formatter)

    ax.set_aspect("equal")


# Plot maps for Podang Caddi, Podang Lompo & Kodingareng Keke
plot_site_map(ax_map_podcaddi, "podcaddi_rates_update", "Podang Caddi")
plot_site_map(ax_map_podlom,   "podlom_rates_update",   "Podang Lompo")
plot_site_map(ax_map_kodkeke,  "kodkeke_rates_update",  "Kodingareng Keke")

fig.tight_layout()

if OUT_PNG:
    fig.savefig(OUT_PNG, dpi=300)
    print("Saved figure to:", OUT_PNG)

plt.show()
